# Nettoyage des données V2

In [20]:
pd.set_option("display.max_columns", None)

### Récupération des données

In [1]:
from scripts.importation import process_all_years_s3

In [2]:
import s3fs
import pandas as pd

In [3]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [4]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(10)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,...,NA,NA,NA,2017,None,None,None,None,None,None
3,3038425,A1,FR,PUBDEM,FR,3038425,20170106,2017-01,Brevet,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,...,NA,NA,NA,2017,None,None,None,None,None,None
4,3038316,A1,FR,PUBDEM,FR,3038316,20170106,2017-01,Brevet,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,...,NA,NA,NA,2017,None,None,None,None,None,None
5,3038399,A1,FR,PUBDEM,FR,3038399,20170106,2017-01,Brevet,PROCEDE ET DISPOSITIF DE COMMANDE D'UNE CONSIG...,...,NA,NA,NA,2017,None,None,None,None,None,None
6,3038381,A1,FR,PUBDEM,FR,3038381,20170106,2017-01,Brevet,DISPOSITIF CAPTEUR PREVU POUR ETRE EMBARQUE DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
7,3038328,A1,FR,PUBDEM,FR,3038328,20170106,2017-01,Brevet,VEHICULE FERROVIAIRE POUR LE NETTOYAGE DES FIL...,...,NA,NA,NA,2017,None,None,None,None,None,None
8,3038251,A1,FR,PUBDEM,FR,3038251,20170106,2017-01,Brevet,PLANCHER PVC EN FIBRES COUPEES ET PROCEDE DE F...,...,NA,NA,NA,2017,None,None,None,None,None,None
9,3038374,A1,FR,PUBDEM,FR,3038374,20170106,2017-01,Brevet,INSTALLATION THERMIQUE DE SECHAGE DE MATIERE P...,...,NA,NA,NA,2017,None,None,None,None,None,None


### Observation des variables

#### **1) Type de brevet**

In [5]:
# Colonne "kind"
data_brevets["kind"].value_counts()

kind
A1    107749
A3      4441
NA       102
B1         3
Name: count, dtype: int64

Le document contient trois types de brevets :

- **A1** : demande de brevet  
- **A3** : demande de certificat d'utilité  
- **B1** : brevet délivré  


**Concernant les types B1 (n = 3)** : cette base recensant les demandes de brevet, elle n'est pas censée contenir un brevet B1.  
Probablement dû à une erreur de classification, nous pouvons donc retirer ces observations.

In [23]:
# Suppression des B1
data_brevets_clean = data_brevets[data_brevets["kind"] != "B1"]

# Vérification qu'on a supprimé 3 observations
data_brevets_clean.shape[0]


112292

**Concernant les types A3 (n = 4 441)** : les certificats d'utilité sont des certificats délivrés par l'INPI qui protègent l'innovation, tout comme les brevets. En revanche la durée de protection est raccourcie (10 ans contre 20 ans) et la procédure d'examen de la demande est allégée. Pour simplifier l'analyse en la rendant plus homogène, nous ne gardons pas ces demandes A3 dans notre base.

In [24]:
# Suppression des A3
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "A3"]

# Vérification : suppression de 4 441 observations (112,292 - 4,441 = 107,851)
data_brevets_clean.shape[0]

107851

**Concernant les NA (n = 102)** : plus d'observations sont nécessaires pour décider de la manière de gérer ces NA.


In [25]:
# Observation des NA
brevets_na = data_brevets[data_brevets["kind"] == "NA"]
brevets_na


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
285,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
606,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
831,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1153,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1543,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28236,NA,NA,NA,NA,None,None,None,None,None,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,N

In [26]:
# Vérification du nombre d'informations réellement présentes
brevets_na["country"].nunique()
brevets_na["invention-title"].nunique()

2

On peut constater que seules deux observations contiennent des informations. Nous pouvons donc supprimer toutes les observations qui ne contiennent que des NA.

In [27]:
# Suppression des observations 100% NA
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "NA"]

#### **2) Numéro du document**

In [28]:
# Vérification que doc_number est unique

nb_number = data_brevets_clean["doc-number"].nunique()
print("Nombre de doc-number uniques :", nb_number)

nb_number2 = data_brevets_clean["publication_doc-number"].nunique()
print("Nombre de publication_doc-number uniques :", nb_number2)

nb_observations = data_brevets_clean.shape[0]
print("Nombre total d'observations :", nb_observations)


Nombre de doc-number uniques : 107748
Nombre de publication_doc-number uniques : 107748
Nombre total d'observations : 107749


Nous pouvons constater qu'il y a une observation qui n'a pas un numéro unique.

In [29]:
doublons = data_brevets_clean[data_brevets_clean.duplicated(subset=["doc-number"], keep=False)]
print(doublons)

      doc-number kind country  status publication_country  \
12476    3050837   A1      FR  PUBDEM                  FR   
58919    3050837   A1      FR     NEW                  FR   

      publication_doc-number publication_date publication_bopinum  \
12476                3050837         20171103             2017-44   
58919                3050837         20171103             2017-44   

      publication_nature                                    invention-title  \
12476             Brevet  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   
58919  Demande française  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   

                  applicant_1_orgname      applicant_1_address-1  \
12476  BOARDING RING READING SAS None  661 CHE DU CHATEAU VALLON   
58919       BOARDING RING READING SAS  661 CHE DU CHATEAU VALLON   

      applicant_1_city applicant_1_postcode applicant_1_country  \
12476        OLLIOULES                83190                  FR   
58919        OLLIOULES         

#### **3) Statut**

La documentation de l'INPI nous apprend que la variable "status" est utilisée pour indiquer quel est l'évènement qui a entraîné la présence du brevet ou du CCP dans le fichier de la semaine. Les 

In [13]:
# Colonne "status"
data_brevets_clean["status"].value_counts()

status
NEW        79065
PUBDEM     28124
AMENDED      560
Name: count, dtype: int64

In [14]:
data_brevets_clean.shape[0]

107749

#### **4) Abstract**
Les abstracts sont les résumés des brevets déposés. Il s'agit pour nous de vérifier qu'ils contiennent bien du texte d'une part, et qu'il n'y a pas de doublons d'autre part.

In [22]:
counts = data_brevets_clean["abstract"].value_counts()
print(counts[counts > 1])

abstract
NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

On peut constater qu'il y a 1166 abstracts en double. On peut tout d'abord supprimer ceux qui contiennent des NA car il semble probable que cela soit du à une erreur dans le dossier original, une demande de brevet sans abstract n'étant pas recevable. Il y en a 339, ce qui n'est pas un nombre trop conséquent au vu de nos plus de 100,000 observations.

In [30]:
data_brevets_clean.groupby("abstract").filter(lambda x: len(x) > 1)


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
48,3038286,A1,FR,PUBDEM,FR,3038286,20170106,2017-01,Brevet,ENSEMBLE COMPRENANT UN GARDE-BOUE POUR VEHICUL...,PEUGEOT CITROEN AUTOMOBILES SA None,ROUTE DE GISY,VELIZY VILLACOUBLAY,78140,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,BERTH,STEPHANE,21 RUE DES PIERRELAIS,CHATILLON,92320,FR,GUILBAUT,MARC,31 RUE DU PRESIDENT KENNEDY,ALFORTVILLE,94140,FR,NA,NA,NA,NA,NA,NA,NA,SERVICE PROPRIETE INDUSTRIELLE\n18 RUE DES FAU...,LA GARENNE COLOMBES,92250,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B62D 25/16 20060101AFI20150703BHFR ...,NA,NA,Ensemble (2) comprenant :,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
76,3038249,A1,FR,PUBDEM,FR,3038249,20170106,2017-01,Brevet,VITRAGE CHAUFFANT A FEUILLE DE VERRE EXTERIEUR...,SAINT-GOBAIN GLASS FRANCE None,18 AVENUE D'ALSACE,COURBEVOIE,92400,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,TONDU,THOMAS,27 RUE DES VERGERS,SULLY,45600,FR,CHAUSSADE,PIERRE,8 ALLEE DU VERGER,ORLEANS,45000,FR,LEGOIS,VINCENT,8 CHEMIN DU PONT CHALUMEAU \nLA PRIEUREE,GERMIGNY DES PRES,45110,FR,NA,DEPARTEMENT PROPRIETE INDUSTRIELLE - 39 QUAI L...,AUBERVILLIERS,93300,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B32B 17/10 20060101AFI20150702BHFR ...,H05B 3/84 20060101ALI20150702BHFR ...,B64C 1/14 20060101ALI20150702BHFR ...,L'invention concerne,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
84,3038325,A1,FR,PUBDEM,FR,3038325,20170106,2017-01,Brevet,PROCEDE DE CRYOCONSERVATION DE CELLULES A VISE...,LABORATOIRE FRANCAIS DU FRACTIONNEMENT ET DES ...,ZA DE COURTABOEUF 3 AV DES TROPIQUES,LES ULIS,91940,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,DE LARICHAUDY,JOFFREY,38 RUE SAVIER,MALAKOFF,92240,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,66 RUE DE LA CHAUSSEE D ANTIN,PARIS CEDEX 09,75440,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,C12N 5/02 20060101AFI20150630BHFR ...,A01N 1/00 20060101ALI20150630BHFR ...,NA,La présente invention se rapporte à une compos...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
131,3038279,A1,FR,PUBDEM,FR,3038279,20170106,2017-01,Brevet,PROCE

In [16]:
data_brevets_clean["abstract"].nunique()


105885

### Tri des variables pertinentes

In [17]:
data_brevets_light = data_brevets[['publication_date', 'invention-title', 'classification_1_text', 'abstract']]
data_brevets_clean.head(5)


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,...,NA,NA,NA,2017,None,None,None,None,None,None
3,3038425,A1,FR,PUBDEM,FR,3038425,20170106,2017-01,Brevet,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,...,NA,NA,NA,2017,None,None,None,None,None,None
4,3038316,A1,FR,PUBDEM,FR,3038316,20170106,2017-01,Brevet,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,...,NA,NA,NA,2017,None,None,None,None,None,None
